<a href="https://colab.research.google.com/github/Vkoundinya/SudokuSolver/blob/main/opencvSudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import cv2
import numpy as np
import imutils
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from sudoku import Sudoku

# New section

In [42]:

def solveSudoku(board):
    solveGrid(0,0,board)
    return board

def solveGrid(row,col,board):
    currentRow=row
    currentCol=col
    if currentCol==len(board[row]):
        currentRow+=1
        currentCol=0
        if currentRow==len(board):
            return True
    if board[currentRow][currentCol]==0:
        return tryFilling(currentRow,currentCol,board)
    return solveGrid(currentRow,currentCol+1,board)

def tryFilling(row,col,board):
    for digit in range(1,10):
        if validPosition(digit,row,col,board):
            board[row][col]=digit
            if solveGrid(row,col+1,board):
                return True
    board[row][col]=0
    return False
                         
        
def validPosition(value,row,col,board):
    rowIsValid = value not in board[row]
    colIsValid = value not in map(lambda r:r[col],board)
    if not rowIsValid or not colIsValid:
        return False
    subRow=row//3
    subCol=col//3
    for rowIdx in range(3):
        for colIdx in range(3):
            rowToCheck=subRow*3+rowIdx
            colToCheck=subCol*3+colIdx
            valueExisting=board[rowToCheck][colToCheck]
            if valueExisting==value:
                return False
    return True
            


In [43]:
def find_board(puzzle):
    gray_image = cv2.cvtColor(puzzle, cv2.COLOR_BGR2GRAY)

    blurred_image = cv2.GaussianBlur(gray_image, (7, 7), 3)
    blurred_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    blurred_image = cv2.bitwise_not(blurred_image, blurred_image)
    kernel = np.array([[0., 1., 0.], [1., 1., 1.], [0., 1., 0.]], np.uint8)
    dialated_image = cv2.dilate(blurred_image, (13,13),8)

    cnts = cv2.findContours(blurred_image, cv2.RETR_LIST,
                            cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    puzzleCnt = None

    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            puzzleCnt = approx
            break

    if puzzleCnt is None:
        raise Exception(("Could not find Sudoku puzzle outline. "
                         "Try debugging your thresholding and contour steps."))

    puzzle1 = four_point_transform(puzzle, puzzleCnt.reshape(4, 2))
    warped = four_point_transform(gray_image, puzzleCnt.reshape(4, 2))
    return puzzle1, warped

Code to Find the Digit if present from Each Cell

In [44]:
def find_digit_in_cell(cell):
    thresh = cv2.threshold(cell, 0, 255,
                           cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    thresh = clear_border(thresh)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_LIST,
                            cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)
    if len(cnts) == 0:
        return None
        # otherwise, find the largest contour in the cell and create a
        # mask for the contour
    c = max(cnts, key=cv2.contourArea)
    mask = np.zeros(thresh.shape, dtype="uint8")
    cv2.drawContours(mask, [c], -1, 255, -1)
    (h, w) = thresh.shape
    percentFilled = cv2.countNonZero(mask) / float(w * h)
    # if less than 3% of the mask is filled then we are looking at
    # noise and can safely ignore the contour
    if percentFilled < 0.03:
        return None
    # apply the mask to the thresholded cell
    digit = cv2.bitwise_and(thresh, thresh, mask=mask)
    return digit

In [45]:
puzzle = cv2.imread('test_image.jpeg')
puzzle = cv2.resize(puzzle, (1400, 800))

puzzleBoard, warped = find_board(puzzle)

#model=keras.models.load_model('digitPredict.h5')
board = np.zeros((9, 9), dtype="int")
# a Sudoku puzzle is a 9x9 grid (81 individual cells), so we can
# infer the location of each cell by dividing the warped image
# into a 9x9 gridß
stepX = warped.shape[1] // 9
stepY = warped.shape[0] // 9
# initialize a list to store the (x, y)-coordinates of each cell
# location
cellLocs = []

for y in range(0, 9):
    # initialize the current list of cell locations
    row = []
    for x in range(0, 9):
        # compute the starting and ending (x, y)-coordinates of the
        # current cell
        startX = x * stepX
        startY = y * stepY
        endX = (x + 1) * stepX
        endY = (y + 1) * stepY
        # add the (x, y)-coordinates to our cell locations list
        row.append((startX, startY, endX, endY))

        cell = warped[startY:endY, startX:endX]
        digit = find_digit_in_cell(cell)
        # verify that the digit is not empty
        if digit is not None:
            roi = cv2.resize(digit, (28, 28))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            pred = model.predict(roi).argmax(axis=1)[0]
            board[y, x] = pred
        cellLocs.append(row)
board =board.tolist()
for row in board:
  print(row)


[8, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 3, 6, 0, 0, 0, 0, 0]
[0, 7, 0, 0, 9, 0, 2, 0, 0]
[0, 5, 0, 0, 0, 7, 0, 0, 0]
[0, 0, 0, 0, 4, 5, 7, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 3, 0]
[0, 0, 1, 0, 0, 0, 0, 6, 8]
[0, 0, 8, 5, 0, 0, 0, 1, 0]
[0, 9, 0, 0, 0, 0, 4, 0, 0]


In [46]:
solutionBoard=solveSudoku(board)
puzzle=Sudoku(3,3,board=solutionBoard)
print(puzzle)



---------------------------
9x9 (3x3) SUDOKU PUZZLE
Difficulty: SOLVED
---------------------------
+-------+-------+-------+
| 8 1 2 | 7 5 3 | 6 4 9 |
| 9 4 3 | 6 8 2 | 1 7 5 |
| 6 7 5 | 4 9 1 | 2 8 3 |
+-------+-------+-------+
| 1 5 4 | 2 3 7 | 8 9 6 |
| 3 6 9 | 8 4 5 | 7 2 1 |
| 2 8 7 | 1 6 9 | 5 3 4 |
+-------+-------+-------+
| 5 2 1 | 9 7 4 | 3 6 8 |
| 4 3 8 | 5 2 6 | 9 1 7 |
| 7 9 6 | 3 1 8 | 4 5 2 |
+-------+-------+-------+

        
